### Data Dictionary

The goal of this exercise is to do Model Development and Validation to find the answer to the Question 4 of the problem statement:

Can a predictive model be built for future prediction of the possibility of complaints of the specific type that you identified in response to Question 1?

Using the best model, you need to predict the number of future complaints (of the Complaint Type that you decided to focus on in Question 1).

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import to_graphviz, plot_importance

%matplotlib inline
sns.set_style('dark')
sns.set(font_scale=1.5)

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score

import feature_engine.missing_data_imputers as mdi
from feature_engine.outlier_removers import Winsorizer

import warnings
warnings.filterwarnings('ignore')

import pickle
from pickle import dump, load

pd.options.display.max_columns= None
#pd.options.display.max_rows = None

### Data Exploration

In [2]:
df = pd.read_csv("parttwo.csv")

In [3]:
df

,ComplaintType,Zipcode,Street,Borough
0,HEAT/HOT WATER,10019.0,WEST 52 STREET,MANHATTAN
1,UNSANITARY CONDITION,11204.0,67 STREET,BROOKLYN
2,HEAT/HOT WATER,11372.0,37 AVENUE,QUEENS
3,HEAT/HOT WATER,10458.0,SOUTHERN BOULEVARD,BRONX
4,APPLIANCE,11209.0,78 STREET,BROOKLYN
...,...,...,...,...
5119322,HEAT/HOT WATER,10029.0,EAST 108 STREET,MANHATTAN
5119323,HEAT/HOT WATER,10461.0,BRUCKNER BOULEVARD,BRONX
5119324,HEAT/HOT WATER,10034.0,SHERMAN AVENUE,MANHATTAN
5119325,HEAT/HOT WATER,10467.0,WEST GUN HILL ROAD,BRONX


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5119327 entries, 0 to 5119326
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ComplaintType  object 
 1   Zipcode        float64
 2   Street         object 
 3   Borough        object 
dtypes: float64(1), object(3)
memory usage: 156.2+ MB


In [5]:
df.shape

(5119327, 4)

In [6]:
df.columns

Index(['ComplaintType', 'Zipcode', 'Street', 'Borough'], dtype='object')

### Data Preprocessing

### Treat Missing Values

In [7]:
df.isnull().sum()

ComplaintType    0
Zipcode          0
Street           0
Borough          0
dtype: int64

### Segment Heat/Hot Water Cases for Bronx

In [8]:
df['Borough'].value_counts()

BROOKLYN         1731202
BRONX            1609837
MANHATTAN        1049360
QUEENS            641741
STATEN ISLAND      87187
Name: Borough, dtype: int64

In [9]:
bronx = df[df['Borough'] == 'BRONX']

In [10]:
bronx

,ComplaintType,Zipcode,Street,Borough
3,HEAT/HOT WATER,10458.0,SOUTHERN BOULEVARD,BRONX
5,HEAT/HOT WATER,10456.0,MORRIS AVENUE,BRONX
10,HEAT/HOT WATER,10459.0,ALDUS STREET,BRONX
16,UNSANITARY CONDITION,10456.0,EAST 164 STREET,BRONX
19,HEAT/HOT WATER,10473.0,BOYNTON AVENUE,BRONX
...,...,...,...,...
5119318,HEAT/HOT WATER,10470.0,EAST 242 STREET,BRONX
5119319,HEAT/HOT WATER,10458.0,BAINBRIDGE AVENUE,BRONX
5119321,UNSANITARY CONDITION,10472.0,EAST 174 STREET,BRONX
5119323,HEAT/HOT WATER,10461.0,BRUCKNER BOULEVARD,BRONX


In [11]:
df2 = pd.read_csv("partfive.csv")

In [12]:
df2

,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,Age,Period
0,122 BRUCKNER BOULEVARD,2340,46.0,0.90,5.0,6.5,1,15000,200.0,1,2.0,2628,2000,6.02,2458,1935,2000,10454,65,85
1,126 BRUCKNER BOULEVARD,752,16.0,0.05,5.0,6.5,4,13770,100.0,2,1.0,272,2000,6.02,2458,1931,1994,10454,63,89
2,138 BRUCKNER BOULEVARD,39375,200.0,1.13,5.0,6.5,10,35000,200.0,1,2.0,2628,2000,6.02,2458,1931,2000,10454,69,89
3,144 BRUCKNER BOULEVARD,12500,85.0,5.00,5.0,6.5,17,2500,100.0,1,5.0,2628,12500,6.02,2458,1931,2001,10454,70,89
4,148 BRUCKNER BOULEVARD,8595,70.0,4.58,5.0,6.5,18,1875,75.0,1,5.0,2628,6876,6.02,1719,1920,2009,10454,89,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89849,GOLD STREET,2340,46.0,0.90,2.0,2.0,100,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85
89850,GOLD STREET,2340,46.0,0.90,3.4,6.5,150,2513,100.0,1,2.0,2628,2000,6.02,2458,1935,2000,10465,65,85
89851,GOLD STREET,2340,46.0,0.90,2.0,2.0,200,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85
89852,GOLD STREET,2340,46.0,0.90,2.0,2.0,8900,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89854 entries, 0 to 89853
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Address     89854 non-null  object 
 1   BldgArea    89854 non-null  int64  
 2   BldgDepth   89854 non-null  float64
 3   BuiltFAR    89854 non-null  float64
 4   CommFAR     89854 non-null  float64
 5   FacilFAR    89854 non-null  float64
 6   Lot         89854 non-null  int64  
 7   LotArea     89854 non-null  int64  
 8   LotDepth    89854 non-null  float64
 9   NumBldgs    89854 non-null  int64  
 10  NumFloors   89854 non-null  float64
 11  OfficeArea  89854 non-null  int64  
 12  ResArea     89854 non-null  int64  
 13  ResidFAR    89854 non-null  float64
 14  RetailArea  89854 non-null  int64  
 15  YearBuilt   89854 non-null  int64  
 16  YearAlter1  89854 non-null  int64  
 17  ZipCode     89854 non-null  int64  
 18  Age         89854 non-null  int64  
 19  Period      89854 non-nul

In [14]:
bs = bronx.sample(n=89854, random_state=0)

In [15]:
bs

,ComplaintType,Zipcode,Street,Borough
3583519,HEAT/HOT WATER,10460.0,MOHEGAN AVENUE,BRONX
786084,GENERAL CONSTRUCTION,10466.0,EAST 224 STREET,BRONX
144876,HEAT/HOT WATER,10451.0,EAST 162 STREET,BRONX
1129806,HEAT/HOT WATER,10458.0,BRIGGS AVENUE,BRONX
5110673,HEAT/HOT WATER,10461.0,PILGRIM AVENUE,BRONX
...,...,...,...,...
3757837,HEAT/HOT WATER,10468.0,EAST 191 STREET,BRONX
1543956,ELECTRIC,10472.0,ELDER AVENUE,BRONX
1247363,HEAT/HOT WATER,10468.0,GRAND AVENUE,BRONX
4391208,GENERAL,10452.0,PLIMPTON AVENUE,BRONX


In [16]:
bs.reset_index(drop=True,inplace=True)

In [17]:
df3 = pd.concat([bs,df2],axis=1)

In [18]:
df3

,ComplaintType,Zipcode,Street,Borough,Address,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,ZipCode,Age,Period
0,HEAT/HOT WATER,10460.0,MOHEGAN AVENUE,BRONX,122 BRUCKNER BOULEVARD,2340,46.0,0.90,5.0,6.5,1,15000,200.0,1,2.0,2628,2000,6.02,2458,1935,2000,10454,65,85
1,GENERAL CONSTRUCTION,10466.0,EAST 224 STREET,BRONX,126 BRUCKNER BOULEVARD,752,16.0,0.05,5.0,6.5,4,13770,100.0,2,1.0,272,2000,6.02,2458,1931,1994,10454,63,89
2,HEAT/HOT WATER,10451.0,EAST 162 STREET,BRONX,138 BRUCKNER BOULEVARD,39375,200.0,1.13,5.0,6.5,10,35000,200.0,1,2.0,2628,2000,6.02,2458,1931,2000,10454,69,89
3,HEAT/HOT WATER,10458.0,BRIGGS AVENUE,BRONX,144 BRUCKNER BOULEVARD,12500,85.0,5.00,5.0,6.5,17,2500,100.0,1,5.0,2628,12500,6.02,2458,1931,2001,10454,70,89
4,HEAT/HOT WATER,10461.0,PILGRIM AVENUE,BRONX,148 BRUCKNER BOULEVARD,8595,70.0,4.58,5.0,6.5,18,1875,75.0,1,5.0,2628,6876,6.02,1719,1920,2009,10454,89,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89849,HEAT/HOT WATER,10468.0,EAST 191 STREET,BRONX,GOLD STREET,2340,46.0,0.90,2.0,2.0,100,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85
89850,ELECTRIC,10472.0,ELDER AVENUE,BRONX,GOLD STREET,2340,46.0,0.90,3.4,6.5,150,2513,100.0,1,2.0,2628,2000,6.02,2458,1935,2000,10465,65,85
89851,HEAT/HOT WATER,10468.0,GRAND AVENUE,BRONX,GOLD STREET,2340,46.0,0.90,2.0,2.0,200,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85
89852,GENERAL,10452.0,PLIMPTON AVENUE,BRONX,GOLD STREET,2340,46.0,0.90,2.0,2.0,8900,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,10465,65,85


In [19]:
df3.columns

Index(['ComplaintType', 'Zipcode', 'Street', 'Borough', 'Address', 'BldgArea',
       'BldgDepth', 'BuiltFAR', 'CommFAR', 'FacilFAR', 'Lot', 'LotArea',
       'LotDepth', 'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea',
       'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1', 'ZipCode', 'Age',
       'Period'],
      dtype='object')

In [20]:
df3.drop(['Zipcode','Street','Borough','Address','ZipCode'],axis=1,inplace=True)

In [21]:
df3

,ComplaintType,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,Age,Period
0,HEAT/HOT WATER,2340,46.0,0.90,5.0,6.5,1,15000,200.0,1,2.0,2628,2000,6.02,2458,1935,2000,65,85
1,GENERAL CONSTRUCTION,752,16.0,0.05,5.0,6.5,4,13770,100.0,2,1.0,272,2000,6.02,2458,1931,1994,63,89
2,HEAT/HOT WATER,39375,200.0,1.13,5.0,6.5,10,35000,200.0,1,2.0,2628,2000,6.02,2458,1931,2000,69,89
3,HEAT/HOT WATER,12500,85.0,5.00,5.0,6.5,17,2500,100.0,1,5.0,2628,12500,6.02,2458,1931,2001,70,89
4,HEAT/HOT WATER,8595,70.0,4.58,5.0,6.5,18,1875,75.0,1,5.0,2628,6876,6.02,1719,1920,2009,89,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89849,HEAT/HOT WATER,2340,46.0,0.90,2.0,2.0,100,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85
89850,ELECTRIC,2340,46.0,0.90,3.4,6.5,150,2513,100.0,1,2.0,2628,2000,6.02,2458,1935,2000,65,85
89851,HEAT/HOT WATER,2340,46.0,0.90,2.0,2.0,200,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85
89852,GENERAL,2340,46.0,0.90,2.0,2.0,8900,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85


In [22]:
# Save to CSV
#df3.to_csv("bronxclassif.csv",index=False)

In [23]:
df3['ComplaintType'].value_counts()

HEAT/HOT WATER          33740
PAINT/PLASTER           10830
PLUMBING                10080
UNSANITARY CONDITION     7876
GENERAL CONSTRUCTION     5952
ELECTRIC                 4047
DOOR/WINDOW              3659
WATER LEAK               3451
NONCONST                 2782
FLOORING/STAIRS          2398
GENERAL                  2299
APPLIANCE                1818
SAFETY                    743
ELEVATOR                   92
OUTSIDE BUILDING           86
STRUCTURAL                  1
Name: ComplaintType, dtype: int64

### Predict top 5 311 Problems (HEAT/HOT WATER, PAINT/PLASTER, PLUMBING, UNSANITARY CONDITION ) Create a new category called Others

In [24]:
df3.replace(to_replace='GENERAL CONSTRUCTION',value='OTHERS',inplace=True)

In [25]:
df3.replace(to_replace='ELECTRIC',value='OTHERS',inplace=True)

In [26]:
df3.replace(to_replace='DOOR/WINDOW',value='OTHERS',inplace=True)

In [27]:
df3.replace(to_replace='WATER LEAK',value='OTHERS',inplace=True)

In [28]:
df3.replace(to_replace='NONCONST',value='OTHERS',inplace=True)

In [29]:
df3.replace(to_replace='FLOORING/STAIRS',value='OTHERS',inplace=True)

In [30]:
df3.replace(to_replace='GENERAL',value='OTHERS',inplace=True)

In [31]:
df3.replace(to_replace='APPLIANCE',value='OTHERS',inplace=True)

In [32]:
df3.replace(to_replace='SAFETY',value='OTHERS',inplace=True)

In [33]:
df3.replace(to_replace='ELEVATOR',value='OTHERS',inplace=True)

In [34]:
df3.replace(to_replace='OUTSIDE BUILDING',value='OTHERS',inplace=True)

In [35]:
df3.replace(to_replace='STRUCTURAL',value='OTHERS',inplace=True)

In [36]:
df3['ComplaintType'].value_counts()

HEAT/HOT WATER          33740
OTHERS                  27328
PAINT/PLASTER           10830
PLUMBING                10080
UNSANITARY CONDITION     7876
Name: ComplaintType, dtype: int64

In [37]:
df3

,ComplaintType,BldgArea,BldgDepth,BuiltFAR,CommFAR,FacilFAR,Lot,LotArea,LotDepth,NumBldgs,NumFloors,OfficeArea,ResArea,ResidFAR,RetailArea,YearBuilt,YearAlter1,Age,Period
0,HEAT/HOT WATER,2340,46.0,0.90,5.0,6.5,1,15000,200.0,1,2.0,2628,2000,6.02,2458,1935,2000,65,85
1,OTHERS,752,16.0,0.05,5.0,6.5,4,13770,100.0,2,1.0,272,2000,6.02,2458,1931,1994,63,89
2,HEAT/HOT WATER,39375,200.0,1.13,5.0,6.5,10,35000,200.0,1,2.0,2628,2000,6.02,2458,1931,2000,69,89
3,HEAT/HOT WATER,12500,85.0,5.00,5.0,6.5,17,2500,100.0,1,5.0,2628,12500,6.02,2458,1931,2001,70,89
4,HEAT/HOT WATER,8595,70.0,4.58,5.0,6.5,18,1875,75.0,1,5.0,2628,6876,6.02,1719,1920,2009,89,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89849,HEAT/HOT WATER,2340,46.0,0.90,2.0,2.0,100,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85
89850,OTHERS,2340,46.0,0.90,3.4,6.5,150,2513,100.0,1,2.0,2628,2000,6.02,2458,1935,2000,65,85
89851,HEAT/HOT WATER,2340,46.0,0.90,2.0,2.0,200,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85
89852,OTHERS,2340,46.0,0.90,2.0,2.0,8900,2513,100.0,1,2.0,2628,2000,1.25,2458,1935,2000,65,85


In [38]:
#df3.to_csv("bronxclassif.csv",index=False)